In [1]:
from collections import OrderedDict
from PyPDF2 import PdfFileWriter, PdfFileReader
from os import listdir
from os.path import isfile, join
from pprint import pprint
import pandas as pd

def _getFields(obj, tree=None, retval=None, fileobj=None):
    """
    Extracts field data if this PDF contains interactive form fields.
    The *tree* and *retval* parameters are for recursive use.

    :param fileobj: A file object (usually a text file) to write
        a report to on all interactive form fields found.
    :return: A dictionary where each key is a field name, and each
        value is a :class:`Field<PyPDF2.generic.Field>` object. By
        default, the mapping name is used for keys.
    :rtype: dict, or ``None`` if form data could not be located.
    """
    fieldAttributes = {'/FT': 'Field Type', '/Parent': 'Parent', '/T': 'Field Name', '/TU': 'Alternate Field Name',
                       '/TM': 'Mapping Name', '/Ff': 'Field Flags', '/V': 'Value', '/DV': 'Default Value'}
    if retval is None:
        retval = OrderedDict()
        catalog = obj.trailer["/Root"]
        # get the AcroForm tree
        if "/AcroForm" in catalog:
            tree = catalog["/AcroForm"]
        else:
            return None
    if tree is None:
        return retval

    obj._checkKids(tree, retval, fileobj)
    for attr in fieldAttributes:
        if attr in tree:
            # Tree is a field
            obj._buildField(tree, retval, fileobj, fieldAttributes)
            break

    if "/Fields" in tree:
        fields = tree["/Fields"]
        for f in fields:
            field = f.getObject()
            obj._buildField(field, retval, fileobj, fieldAttributes)

    return retval


def get_form_fields(infile):
    infile = PdfFileReader(open(infile, 'rb'))
    fields = _getFields(infile)
    return OrderedDict((k, v.get('/V', '')) for k, v in fields.items())

In [2]:
def cleanAndReorderDict(oDict:OrderedDict) -> OrderedDict:
    for key,value in oDict.items():
        
        if('/True' in value):
            value=True
        if('/False' in value):
            value=False
        print(key,value)

In [6]:

def getAllForms(mypath):
    allForms = []
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    for currentFile in onlyfiles:
        if '.pdf' in currentFile:
            allForms.append(mypath + currentFile)
    return allForms

In [16]:
labPath= '../test-forms/labor_covid19/'
formsLabList = getAllForms(labPath)
for labForm in formsLabList:
    labDict=get_form_fields(labForm)
    pprint(labDict)

OrderedDict([('Nachweis_Tag', ''),
             ('Nachweis_Monat', ''),
             ('Nachweis_Jahr', ''),
             ('Entnahme_Tag', ''),
             ('Entnahme_Monat', ''),
             ('Entnahme_Jahr', ''),
             ('Nasenrachenabstrich', ''),
             ('Mundrachenabstrich', ''),
             ('Sputum', ''),
             ('Endotracheale_Absaugung', ''),
             ('Broncho-alveolaere_Lavage', ''),
             ('Blut_Serologie', ''),
             ('andere', ''),
             ('Andere_Entnahme', ''),
             ('E-gene-target', ''),
             ('ELISA-SARS-CoV-2-IgM', ''),
             ('RdRp-gene-target', ''),
             ('ELISA-SARS-CoV-2-IgG', ''),
             ('N-gene-target', ''),
             ('ELISA-SARS-CoV-2-IgA', ''),
             ('Neue-Sequenzierungsmethode', ''),
             ('IFA-SARS-CoV-2-IgM', ''),
             ('Neue Sequenzierungsmethode_Beschreibung', ''),
             ('IFA-SARS-CoV-2-IgG', ''),
             ('Sequenz-analog-SARS-CoV-2'

In [19]:
meldungPath='../test-forms/covid19/'
formsMeldungList = getAllForms(meldungPath)
meldungDictList=[]
for meldForm in formsMeldungList:
    meldDict=get_form_fields(meldForm)
    meldungDictList.append(meldDict)
    pprint(meldDict)

OrderedDict([('Geschlecht', '/w'),
             ('Telefonnummer', '030 853 99 86'),
             ('Kanton', 'BE'),
             ('Geburtstag_Monat', '11'),
             ('Geburtstag_Jahr', '2020'),
             ('PLZ_Wohnort', '3003 Bern'),
             ('Name_Vorname', 'Cow, Holy'),
             ('Geburtstag_Tag', '12'),
             ('Strasse_Hausnummer', 'Weidestrasse 112'),
             ('Nationalitaet', '/Andere'),
             ('Andere_Nationalitaet', 'deutsch'),
             ('Wohnsitz_Land', ''),
             ('Fieber', '/True'),
             ('Husten', '/True'),
             ('Atembeschwerden', '/True'),
             ('Pneumonie', '/True'),
             ('ARDS', '/True'),
             ('Andere Symptome', '/True'),
             ('Durchfall', '/True'),
             ('Andere_Symptome_Text', 'Kopfschmerz'),
             ('Manifestation_Jahr', '2020'),
             ('Manifestation_Tag', '16'),
             ('Manifestation_Monat', '03'),
             ('Keine_Symptome', ''),
        

In [19]:
cleanAndReorderDict(meldDict)

Kanton BE
Telefonnummer 030 954 63 72
Fieber /True
Husten /True
Atembeschwerden /True
Durchfall /True
ARDS /True
Pneumonie /True
Geburtstag_Tag 12
Geburtstag_Monat 11
Geburtstag_Jahr 1989
Andere Symptome /True
Keine_Symptome 
Diabetes /Yes
Herz-Kreislauf /Yes
Immunsuppression /Yes
Hypertonie /Yes
Atemwegserkrankung /Yes
Keine 
Krebs /Yes
Andere_Symptome_Text Kopfschmerz
Andere /Yes
Referenzlabor /Yes
Andere_Grunderkrankungen starke Raucherin
Manifestation_Tag 18
Manifestation_Monat 03
Manifestation_Jahr 2020
Anderes_Labor 
Entnahme_Tag 19
Entnahme_Monat 03
Entnahme_Jahr 2020
IPS 
Beatmung 
Anderes_Labor_text 
Eintrittsdatum_Tag 20
Eintrittsdatum_Monat 03
Eintrittsdatum_Jahr 2020
TF /Yes
Spital b'Inselspital Bern\rTel 030 548 44 44'
TD_Tag 22
TD_Monat 03
TD_Jahr 2020
Exposition_Tag 13
Exposition_Monat 03
Exposition_Jahr 2020
Von_Tag 06
Von_Monat 03
Von_Jahr 2020
Bemerkungen Keine Bemerkungen
Bis_Tag 16
Bis_Monat 03
Bis_Jahr 2020
Andere_Kontakte_text 
Datum_Tag 23
KA_Datum_Tag 19
Datum_M

In [18]:
df=pd.DataFrame(meldungDictList)
df.head()

,Geschlecht,Telefonnummer,Kanton,Geburtstag_Monat,Geburtstag_Jahr,PLZ_Wohnort,Name_Vorname,Geburtstag_Tag,Strasse_Hausnummer,Nationalitaet,...,Arzt,Datum_Tag,Datum_Monat,Datum_Jahr,Kantonsarzt_Massnahmen_Text,Kantonsarzt_Massnahmen,Kantonsarzt_Bemerkungen,KA_Datum_Tag,KA_Datum_Monat,KA_Datum_Jahr
0,/w,030 853 99 86,BE,11,2020,3003 Bern,"Cow, Holy",12,Weidestrasse 112,/Andere,...,b'Arzt\rHinterkappellenring 111\r3032 Hinterka...,22,03,2020,,/False,KA,20,03,2020
1,/w,030 853 99 86,BE,11,2020,3003 Bern,"Cow, Holy",12,Weidestrasse 112,/Andere,...,b'Arzt\rHinterkappellenring 111\r3032 Hinterka...,22,03,2020,,/False,KA,20,03,2020


In [21]:
meldungenCsv='covid19.csv'
df.to_csv(meldungPath + 'csv/' + meldungenCsv,sep=';')